In [ ]:
https://codereview.stackexchange.com/questions/125286/moving-around-in-a-2d-grid


In [18]:
import pandas as pd

In [67]:
import numpy as np

In [68]:
import random

In [9]:
import Queue

In [ ]:
class Board(list):

    def __str__(self):
        return "\n".join(" ".join(row) for row in self)

In [ ]:
#create randomized jobs

In [4]:
columns= ['PickupX', 'PickupY', 'DropoffX', 'DropoffY']

In [23]:
DFRequests = pd.DataFrame(np.random.randint(4, size=(101, 4)), columns= columns)

In [24]:
DFRequests

,PickupX,PickupY,DropoffX,DropoffY
0,1,2,0,3
1,3,3,1,3
2,3,1,3,0
3,0,0,1,0
4,2,2,1,3
5,2,2,2,2
6,0,2,2,0
7,3,3,1,0
8,2,0,2,3
9,2,3,2,3


In [25]:
#export to CSV
DFRequests.to_csv('Random_Requests.csv')

In [106]:
##Create simulation

class Location():
    #create location, compare to other location
    
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
    
    def __str__(self):
        return "[%s:%s]" % (self.X, self.Y)
    
    def distance_formula(self, other):
        #formula to find the distance
        
        distance = abs(self.X - other.X) + abs(self.Y - other.Y)
        return distance
    

In [114]:
location1 = Location(2,3)
print location1

[2:3]


In [3]:
other_location = Location(0,1)
print other_location

[0:1]


In [115]:
location1.distance_formula(other_location)

4

In [116]:
class Request():
    "assign each request two locations"
    
    def __init__(self, loc1, loc2):
        self.start = loc1
        self.end = loc2
        self.request = (self.start, self.end)
        self.request_distance = Location.distance_formula(self.start, self.end)
        
    def __str__(self):
        
        return "Start: %s  End: %s" % (str(self.start), str(self.end))
        
    

In [86]:
# print Request(location1, other_location)

Start: [2:3]  End: [0:1]


In [108]:
class iterate_Log():
    '''read Log line by line'''
    import Queue
    
    def __init__(self, CSV):
        self.log = pd.read_csv(CSV, index_col=0)
        self.lengthrequestlist = len(self.log.index)
        
        self.chain = Queue.Queue()

        #put items at the end of the queue
        for x in range(self.lengthrequestlist):
            self.chain.put(self.this_request(x))
    
    def this_request(self, row):
        self.row = row
        self.pickuploc = Location(self.log.iloc[row]['PickupX'], self.log.iloc[row]['PickupY'])
        self.dropoffloc = Location(self.log.iloc[row]['DropoffX'], self.log.iloc[row]['DropoffY'])
        self.read_request = Request(self.pickuploc, self.dropoffloc)
        return self.read_request


    def draw_request(self):
        '''assigns request'''
        r = self.chain.get()
        return r
        
    def assign_request(self, request):
        "puts request back in log or will be utilized to put request in employee queue"
        self.chain.put(request)
        
    def current_queue(self):
        d = self.chain.queue
        for x in d:
            print str(x)

        
       
                           

In [36]:
###NEXT STEP ASSIGN JOBS to EMPLOYEE queue

In [117]:
#testing
logtest = iterate_Log('Random_Requests.csv')
print logtest.draw_request()
logtest.draw_request()
logtest.draw_request()

Start: [1:2]  End: [0:3]


<__main__.Request instance at 0x10d33c638>

In [109]:
#create a class that acts like an employee
class Employee(iterate_Log):
    #inherits methos from iterate_Log
    def __init__(self):
        self.chain = Queue.Queue()
        self.currentloc = Location(0,0)
        self.employeedistance = 0
        
    def distance_traveled(self):
        d = self.chain.queue
        for x in d:
            self.employeedistance += Location.distance_formula(self.currentloc, x.start)
            self.employeedistance += x.request_distance
            self.currentloc= x.end
        return self.employeedistance
        

In [110]:
#TESTING
queue = Queue.Queue()
queue.put(15)
queue.put(30)
d = queue.queue
print(d)
print len(d)
# for n in list(queue.):
#     print "This time, it's: "+ str(n)


deque([15, 30])
2


In [118]:
#testing
log = iterate_Log('Random_Requests.csv')
employee1 = Employee()

req = log.draw_request()
print req
employee1.assign_request(req)
print "another one"
req = log.draw_request()
print req
employee1.assign_request(req)

print "QUEUE"
print employee1.current_queue()

print employee1.distance_traveled()
 



Start: [1:2]  End: [0:3]
another one
Start: [3:3]  End: [1:3]
QUEUE
Start: [1:2]  End: [0:3]
Start: [3:3]  End: [1:3]
None
10


In [120]:
class Typ_Day():
    '''assign jobs to employees'''
    def __init__(self, CSV):
        self.newlog = iterate_Log(CSV)
        
        self.employeeOne= Employee()
        self.employeeTwo= Employee()
        
    #def travel_dictionary(self):
        
        
        
    def jobs_to_employees(self):
        while len(self.newlog.chain.queue) >0:
            next_req = self.newlog.draw_request()
            #accomodate for time frame
            #for more than one employee it may be best to use a different condition method such as dictionary
            if self.employeeOne.distance_traveled() <= self.employeeTwo.distance_traveled():
                self.employeeOne.assign_request(next_req)
            else:
                self.employeeTwo.assign_request(next_req)
        ###ADD condition where if the distance from the next in the queue 
        return "all jobs assigned"
    
    def Total_travel(self):
        travel = int(self.employeeOne.distance_traveled()) + int(self.employeeTwo.distance_traveled())
        print "employee One"
        print self.employeeOne.distance_traveled()
        print "employee Two"
        print self.employeeTwo.distance_traveled() 
        return travel


In [126]:
Day1 = Typ_Day('Random_Requests.csv')
Day1.jobs_to_employees()
print Day1.Total_travel()
print "employee One"
print Day1.employeeOne.distance_traveled() 
print "employee Two"
print now() + Day1.employeeTwo.distance_traveled()

employee One
13333
employee Two
13313
26112
employee One
13611
employee Two


NameError: name 'now' is not defined

In [100]:
# class read_Log():
#     '''read Log line by line'''
    
#     def __init__(self, CSV, row):
#         self.row = row
#         log = pd.read_csv(CSV, index_col=0)
#         self.pickuploc = Location(log.iloc[row]['PickupX'], log.iloc[row]['PickupY'])
#         self.dropoffloc = Location(log.iloc[row]['DropoffX'], log.iloc[row]['DropoffY'])
#         self.read_request = (self.pickuploc, self.dropoffloc)

            
            
#     def __str__(self):
#         results = []
#         if self.row > len(log.index):
#             return "row number does not exist"
#         for loc in self.read_request:
#             results.append(str(loc))
#         return str(results)
                           

In [63]:
print read_Log('Random_Requests.csv',3)

['[0:0]', '[1:0]']


In [ ]:
SET CSV on highe level

Read first two lines of Csv
Then one line when repository is 2 locations or less
Each line is a list
First request and second request
Create locations with first two and last two numbers in each line

Start with first request assigned to employee
Locations are assigned to employee queue
First location assigned to employee as current location and removed from queue
Difference found between next location (first location in queue)
Add to employee distance traveled
Second location assigned to employee and removed from queue
Load in next request to queue if list queue is empty
Repeat difference and add distance traveled
Should repeat twice since the queue will not be empty

*employee can't be assigned to request because  it wouldn't capture the distance traveled between requests

To add multiple employees
Compare distance traveled (use as time) amongst employees. Whoever has the least time is assigned the next request

In [ ]:
#     def queue(self):
#         self.all_request_log = Queue.Queue()

#         #put items at the end of the queue
#         for x in range(self.lengthrequestlist):
#             self.all_request_log.put(logtest.this_request(x))

#         #remove items from the head of the queue 
#         while not self.all_request_log.empty():
#             self.all_request_log.get()

In [89]:
# class Log():
#     '''Create Log'''
    
#     def __init__(self):
#         self.pickuploc = []
#         self.dropoffloc = []
#         for i in range(100):
#             pickXrand = random.randint(1,4)
#             pickYrand = random.randint(1,4)
#             dropXrand = random.randint(1,4)
#             dropYrand = random.randint(1,4)
#             pickup = Location(pickXrand, pickYrand)
#             dropoff = Location(dropXrand, dropYrand)
#             self.pickuploc.append(pickup) 
#             self.dropoffloc.append(dropoff)
            
            
#     def __str__(self):
#         results = []
#         for loc in self.pickuploc:
#             results.append(str(loc))
#         return '\n'.join(results)
                           
    

In [ ]:
# def doleOut(queue, workers):
#     for worker,task in itertools.izip(itertools.cycle(workers),queue):
#         yield worker,task

In [ ]:
# test = pd.read_csv('Random_Requests.csv', index_col=0)
# lengthrequestlist = len(test.index)



#put items at the end of the queue
# for x in range(lengthrequestlist):
#     queue.put(logtest.this_request(x))

In [54]:


# class Request():
#     #create request
    
#     def __init__(self, Start= (0,0), End= (0,0)):
#     #set locations as start and end (can be pickup or drop off depending \
#     #upon point in process
#         self.Start = 
#         self.End = 
    
#     def __str__(self):
#         #     def distance_formula(self):
#             #formula to find the distance
        
#         distance = abs(self.PickupX - self.DropoffX) + abs(self.PickupY - self.DropoffY)
        
        
#         #print string for request location
#         return "Pick Up Location: [%s:%s] \nDropoff Location: [%s:%s] \nDistance traveled from Pickup to dropoff is %s"\
#                         % (self.PickupX, self.PickupY, self.DropoffX, self.DropoffY, distance)
    

    

SyntaxError: invalid syntax (<ipython-input-54-51e1731b6316>, line 8)